## Scope
In this notebook we give a very brief tutorial which focuses on the mnms python user interface. We'll use a downgraded version of the `d6` region so that all our operations are fast, even though you will probably never use that sim. This way you know how to use mnms in your own analysis code.

## Tutorial

In mnms, we always need to generate a "sqrt-covariance" matrix as a first step; only then can we generate sims:

In [ ]:
from mnms import noise_models as nm, utils
from soapack import interfaces as sints

# load the DR5 data model in order to be able to use deep6 (qid = 'd6') data
dm = sints.DR5()

Create a "TiledNoiseModel" instance, and a "WaveletNoiseModel" instance:

In [ ]:
tnm = nm.TiledNoiseModel('d6', data_model=dm, downgrade=8, mask_version='masks_20200723', 
                        union_sources='20210209_sncut_10_aggressive', notes='my_first_model')

In [ ]:
wnm = nm.WaveletNoiseModel('d6', data_model=dm, downgrade=8, mask_version='masks_20200723',
                            union_sources='20210209_sncut_10_aggressive', notes='my_first_model')

Now we need to make our noise model. Because they are the slowest step to make, and take up a lot of space, by default they are written to disk. The idea is that outside of testing/development, you only need to ever run this command once (per data-release, array-set, other parameters, etc):

In [ ]:
tnm.get_model()

In [ ]:
wnm.get_model()

To save you from accidentally regenerating the exact same model again, the argument `check_on_disk` is `True` by default. If I rerun `get_model`, it will find that the model is on-disk, and return nothing (you can also keep the model in memory, stored as an instance attribute, if you want, via `keep_model=True`):

In [ ]:
tnm.get_model()

In [ ]:
wnm.get_model()

That was fast, and that's because the models are on-disk:

In [ ]:
ls /scratch/gpfs/zatkins/data/ACTCollaboration/mnms/covmats/*my_first_model*

Note, this trick won't work if you give a model a different `notes` parameter. E.g., `my_second_model` is considered a totally distinct noise model from `my_first_model`, even if all the "scientific" parameters are the same.

Now we can make sims. Sims are always made of single splits at a time. To do so, we must load the "sqrt-covariance" matrix from disk, so that we can sample from it, so make sure to have enough memory on-hand to do this. These files are only loaded once -- they are stored in the object instance, so that future calls to `get_sim` do not have to load from disk. There is also a similar `check_on_disk` parameter to prevent users from generating the same sim twice (by default this is True).

By default, these are ***not*** saved to disk (but they can be, if desired). We need to tell `get_sim` which split and map number we want to make. The map number is used in getting a unique random seed (and if the sims is written to disk, will be stored in the filename):

In [ ]:
tmap_s3_n123 = tnm.get_sim(3, 123, verbose=True)

In [ ]:
wmap_s3_n123 = wnm.get_sim(3, 123, verbose=True)

As noted in the README, these sims always have shape (num_arrays, num_splits, num_pol, ny, nx), even if some of these axes have dimension 1. Because we generate sims per split, `num_splits=1` always. Let's take a look! 

In [ ]:
utils.eshow(tmap_s3_n123, colorbar=True)
utils.eshow(wmap_s3_n123, colorbar=True)